In [9]:
import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

from utils import *
import utils
from consistency_eval import *

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass

In [10]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
%env CUDA_VISIBLE_DEVICES=5

Using GPU: cuda:5
env: CUDA_VISIBLE_DEVICES=5


In [11]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [12]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Tianyi-Lab/Personas")
personas = ds['train']['Llama-3.1-70B-Instruct_descriptive_persona']

In [13]:
np.random.seed(0)
random_indices = np.random.choice(len(personas), size=200, replace=False).astype(int)
personas_updated = np.array(personas)[random_indices]

In [14]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [15]:
import json
import shutil

def get_text_after_colon(agent_name, text):
    """
    Extracts only the target agent's first utterance.
    If other agents speak in the same completion, they are removed.
    """
    pattern = rf"{agent_name}:\s*(.*?)(?:\n|[A-Z][a-z]+:|$)"  # matches up to next speaker or newline
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # fallback: remove all leading speaker tags and return first sentence
        text = re.sub(r'^([A-Za-z]+:\s*)+', '', text)
        return text.strip().split('\n')[0]  # return first line as fallback
        
# Step 1: Read existing config
with open('config/chit-chat/prompts.json', 'r') as f:
    config_prompt = json.load(f)

# Step 2: Overwrite values
config_prompt['dialogue_prompt'] = "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is meeting the other for the first time hoping to get to know the other better. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY% Your conversation so far is below, and it is your turn to respond to %LISTENER_ROLE%. Please be very brief, responding in 2 sentences or less, as this is a natural dialogue, not an essay. DO NOT REPEAT PREVIOUS TALKING POINTS, AS THIS WILL HURT YOUR CREDIBILITY. %CONVERSATION%%SPEAKER_ROLE%: "
config_prompt['eval_prompts']['prompt_consistency'] = "For the following line spoken by %SPEAKER_ROLE%, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%. Note that it is okay for %SPEAKER_ROLE% to provide details not in their background, but these details cannot be inconsistent with te background. %SPEAKER_ROLE%'s background is: %SPEAKER_BACKSTORY% %SPEAKER_ROLE% spoke the following line: %SPEAKER_LINE% Answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if the line does not contradict the provided background of %SPEAKER_ROLE%, followed by 1 sentence of reasoning. Be very strict in your assessment but remain accurate."

# Step 3: Backup
with open('config/chit-chat/prompts.json', 'w') as f:
    json.dump(config_prompt, f, indent=4)

In [16]:
# personas_new = []

# for persona in personas_updated:
#     name_prompt = "Given the following biography below, parse the name of the person. \nOnly output the name.\n" + persona + "\nName: "
#     name = completion_create(config['agent1_model'], config, name_prompt)
#     name = get_text_after_colon("Name", name)
#     print(name)    
#     personas_new.append({'persona': persona, 'name': name})

# with open('config/chit-chat/personas.json', 'w') as f:
#     json.dump(personas_new, f, indent=4)

In [17]:
with open("config/chit-chat/personas.json", "r") as f:
    personas_updated = json.load(f)

In [19]:

# choose config file to load into global config dictionary in utils.py
with open("./config/chit-chat/Llama-3.1-8B-Instruct.json", 'r') as f:
    config_llama = json.load(f)

config_llama["eval_model"] = "Llama-3.1-70B-Instruct.json"

for key, value in config_llama.items():
    config[key] = value

# this modifies the global prompts dictionary in utils.py
with open('config/chit-chat/prompts.json', 'r') as f:
        new_prompts = json.load(f)

for key, value in new_prompts.items():
    prompts[key] = value

In [58]:
import os
from datetime import datetime

# Create timestamped directory
current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"data/anthology/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# File to write output to
write_file = os.path.join(output_dir, f"{config['agent1_model']}_{config['seed']}_1.json")

index_offset = load_stats_file(write_file)

conversations = []
for p_dict1, p_dict2 in tqdm(np.array(personas_updated[:20]).reshape(-1, 2)):
    reset_stats()
    conversation = generate_conversation(
        p_dict1["persona"], 
        p_dict2["persona"], 
        p_dict1["name"], 
        p_dict2["name"], 
        pturn=1
    )
    print(conversation)
    conversations.append(conversation)
    stats['index'] = index_offset
    stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    write_stats(write_file)
    index_offset += 1

written!!


  0%|                                                                                                                    | 0/10 [00:00<?, ?it/s]

gpt-4o-mini


 10%|██████████▊                                                                                                 | 1/10 [00:09<01:25,  9.52s/it]

{'task_name': 'Chit-Chat', 'P1': '"Meet 49-year-old Alexandra \\"Alex\\" Thompson, a vibrant and determined woman living in Charleston, South Carolina. Born and raised in the heart of the Lowcountry, Alex embodies the warmth and hospitality that her home state is famous for. As a white woman, she has had her share of privileges, but she\'s also faced challenges that have shaped her into the strong, independent individual she is today.\\n\\nAlex grew up in a traditional Southern family, where her parents valued hard work, education, and community involvement. Her father was a high school history teacher, and her mother worked as a librarian, instilling in Alex a love for learning and a strong sense of social responsibility. She has two younger brothers, both of whom have moved away from South Carolina to pursue their careers.\\n\\nAfter graduating from the University of South Carolina with a degree in environmental science, Alex worked for several years as a conservation biologist for t

...

{'task_name': 'Chit-Chat', 'P1': '"Meet Alexandra \\"Alex\\" Thompson, a 48-year-old White female living in the heart of Atlanta, Georgia. Born on a crisp autumn morning in October 1975, Alex has grown to be a proud Georgian with a rich Southern drawl and an infectious warmth that instantly puts people at ease. She is a product of the Peach State\'s diverse cultural landscape, having grown up in a small town near the Blue Ridge Mountains, where her family has lived for generations.\\n\\nAlex\'s early life was marked by a strong sense of community and tradition. Her parents, both high school sweethearts, owned a quaint antique shop in the town square, where Alex spent countless hours listening to stories of the past and learning the value of hard work. Her father, a skilled woodworker, would often take her on walks through the nearby woods, teaching her about the local flora and fauna, and instilling in her a deep love for nature.\\n\\nAfter graduating from the University of Georgia wit

 90%|█████████████████████████████████████████████████████████████████████████████████████████████████▏          | 9/10 [01:24<00:08,  8.90s/it]

{'task_name': 'Chit-Chat', 'P1': '"Meet Elena Moreno, an 87-year-old Hispanic woman living in Las Vegas, Nevada. Born and raised in a small town in New Mexico, Elena moved to Nevada in her early 50s to join her siblings who had already made the state their home. She is the youngest of seven children, and her upbringing in a large, close-knit family significantly influenced her strong family values and warm hospitality. Elena\'s parents were both immigrants from Mexico, and Spanish is her first language, although she speaks English fluently as well.\\n\\nElena has short, curly silver hair, often styled in a neat bob, and her bright hazel eyes sparkle with warmth and kindness. She stands at about 5\'2\\" and has a petite yet energetic build, often moving with the help of a stylish wooden cane adorned with colorful, hand-carved flowers \\u2013 a gift from her late husband. Elena\'s style is a vibrant mix of traditional and modern elements, reflecting her rich cultural heritage and her lov

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:36<00:00,  9.61s/it]

{'task_name': 'Chit-Chat', 'P1': '"Meet Ethan Thompson, a 38-year-old White male living in Ames, Iowa. Ethan is a first-generation college graduate, having earned a Bachelor\'s degree in Agricultural Engineering from Iowa State University. He grew up on a small farm just outside of Des Moines, where his family raised livestock and grew corn and soybeans. Ethan\'s upbringing instilled in him a strong work ethic and an appreciation for the land.\\n\\nEthan currently works as a sales engineer for a company that specializes in precision agriculture technology. He spends most of his days traveling across the Midwest, meeting with farmers and demonstrating the latest innovations in farm equipment and software. Ethan\'s job allows him to combine his passion for agriculture with his technical expertise, and he takes pride in helping farmers increase their efficiency and productivity.\\n\\nEthan is married to his high school sweetheart, Rachel, who works as a librarian at the local elementary s

In [16]:
with open('data/anthology/exp/04.15.25/Llama-3.1-8B-Instruct_0_1.json', 'r') as f:
    conversations = json.load(f)
    
for conversation in tqdm(conversations):
    eval_prompt_consistency(conversation)
    eval_prev_line_consistency(conversation)

write_stats(write_file)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5094.50it/s]


In [ ]:
exp_folder = './data/anthology/exp'
# config['filename'] = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/data/persona/exp/gpt-4o-mini_1.json'
config['filename'] = '~.consistency_LLMs/data/anthology/exp/gpt-4o-mini_10.json'
if config['filename']:
    run_metrics(config['filename'])
else:
    for filename in glob.glob(f'{exp_folder}/*.json'):
        run_metrics(filename)

In [ ]:
print(personas_updated[0])